In [1]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from copy import deepcopy
from matplotlib.colors import Normalize
import matplotlib
from matplotlib.ticker import ScalarFormatter
# import scienceplots
# plt.style.use(['science','ieee'])
# plt.style.use(['science','~/.config/matplotlib/stylelib/ingo.mpl'])
plt.style.use('~/.config/matplotlib/stylelib/ingo.mplstyle')
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# cm = plt.get_cmap('tab20')
# cm = plt.get_cmap('Dark2')
# for color in plt.cycler("color", cm.colors):
    # print(matplotlib.colors.to_hex(color['color']))

## Load data

In [2]:
levels = [0,1,2,3,4,5,6]
slices = []
for level in levels:
    slice = []
    filename = '../output_cluster/F/streamlines/lvl{}/out3d.0.vtk'.format(str(level))
    reader = pv.get_reader(filename)
    mesh = reader.read()
    slice.append(mesh.slice(normal = [0,1,0]))
    slice.append(mesh.slice(normal = [0,1,0]))
    slices.append(slice)

## Extract and filter data

In [3]:
selection_criteria = [lambda X: X[:,0] < 0., lambda X: X[:,0] > 0.]
plane_x = [0, 0]
Nx = 11
Nz = 10
vmin = np.inf
vmax = -np.inf
X = []
U = []
V = []
W = []
speed = []
grid = []
for level in levels:
    xx = []
    uu = []
    vv = []
    ww = []
    ss = []
    gg = []
    for i_slice in range(len(selection_criteria)):
        slice= slices[level][i_slice]
        x = slice.points
        u = slice.point_data['0']
        v = slice.point_data['1']
        w = slice.point_data['2']
        selection = selection_criteria[i_slice](x)
        x = x[selection]
        u = u[selection]
        v = v[selection]
        w = w[selection]
        sort_order = np.lexsort((x[:,2], x[:,1], x[:,0]), axis=0)
        x = x[sort_order]
        u = u[sort_order]
        v = v[sort_order]
        w = w[sort_order]

        # I need to resample to a uniform grid in order to use streamlines.
        dx = (x[:,plane_x[i_slice]].max() - x[:,plane_x[i_slice]].min())/Nx
        dz = (x[:,2].max() - x[:,2].min())/Nz
        # gX, gY = np.meshgrid(np.linspace(x[:,0].min()+dx/2, x[:,0].max()-dx/2, Nx), np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz/2, Nz))
        gX, gY = np.meshgrid(np.linspace(x[:,plane_x[i_slice]].min(), x[:,plane_x[i_slice]].max(), Nx), np.linspace(x[:,2].min(), x[:,2].max()-dz, Nz))

        # set x to unit cube
        # gX, gY = np.meshgrid(np.linspace(0,1, Nx), np.linspace(0,1, Nz))
        # x = np.array([gX.flatten(), gY.flatten()]).T
        # sort_order = np.lexsort((x[:,1], x[:,0]), axis=0)
        # x = x[sort_order]

        # TODO THIS IS not close enough to the bounday
        # gX, gY = np.meshgrid(np.linspace(x[:,plane_x[i_slice]].min()+dx, x[:,plane_x[i_slice]].max()-dx, Nx), np.linspace(x[:,2].min()+dz, x[:,2].max()-dz, Nz))
        g = np.stack((gX, gY), axis=-1)
        u = griddata(x[:,[plane_x[i_slice],2]], u, (gX, gY), rescale=False)
        v = griddata(x[:,[plane_x[i_slice],2]], v, (gX, gY), rescale=False)
        w = griddata(x[:,[plane_x[i_slice],2]], w, (gX, gY), rescale=False)
        s = np.sqrt([u, v][plane_x[i_slice]]**2 + w**2)

        assert (not np.isnan(u).any())
        assert (not np.isnan(v).any())
        vmin = min(vmin, s[~np.isnan(s)].min())
        vmax = max(vmax, s[~np.isnan(s)].max())
        xx.append(deepcopy(x))
        uu.append(deepcopy(u))
        vv.append(deepcopy(v))
        ww.append(deepcopy(w))
        ss.append(deepcopy(s))
        gg.append(deepcopy([gX, gY]))
    X.append(deepcopy(xx))
    U.append(deepcopy(uu))
    V.append(deepcopy(vv))
    W.append(deepcopy(ww))
    grid.append(deepcopy(gg))
    speed.append(deepcopy(ss))
        

In [4]:
fig, ax = plt.subplots(4,2, layout='constrained')
N_streamlines = 10
streamlines = []
for row in range(4):
    for col in range(2):
        x = X[row][col]
        u = U[row][col]
        v = V[row][col]
        w = W[row][col]
        s = speed[row][col]
        g = grid[row][col]
        dx = (x[:,0].max() - x[:,0].min())/Nx
        # dz = (x[:,2].max() - x[:,2].min())/Nz
        dz = (x[:,2].max() - x[:,2].min())/Nz
        seed_points = np.array([x[:,plane_x[col]].mean() * np.ones(N_streamlines), np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz, N_streamlines)])
        # seed_points = np.array([-3.9 * np.ones(N_streamlines), np.linspace(x[:,2].min()+2*dz, x[:,2].max()-2*dz, N_streamlines)])
        # seed_points = np.array([0.5 * np.ones(N_streamlines), np.linspace(0, 1, N_streamlines)])
        # seed_points = np.array([x[:,plane_x[col]].mean() * np.ones(N_streamlines), np.linspace(x[:,2].min()+dz, x[:,2].max()-dz, N_streamlines)])
        # ax[row, col].scatter(seed_points[0,:], seed_points[1,:])

        ax[row, col].xaxis.set_ticks([])
        ax[row, col].yaxis.set_ticks([])
        ax[row, col].xaxis.set_ticklabels([])
        ax[row, col].yaxis.set_ticklabels([])
        ax[row, col].set_xlabel('')
        if col == 0:
            ax[row, col].set_ylabel('level {} \n H'.format(str(row)))
        else:
            ax[row, col].set_ylabel('')
        strm = ax[row, col].streamplot(g[0],g[1],[u, v][plane_x[col]],w, broken_streamlines=False, start_points=seed_points.T, color=100*s.reshape((u.shape[0], u.shape[1])), norm = Normalize(vmin=100*vmin, vmax=100*vmax, clip=False))
        streamlines.append(strm)
        # ax[row, col].set_ylim(x[:,1].min(),x[:,1].max())


ax[3, 0].set_xlabel('outer wall$\leftrightarrow$inner wall')
ax[3, 1].set_xlabel('inner wall$\leftrightarrow$outer wall')
ax[0,0].set_title('$90^\circ$ cross-section')
ax[0,1].set_title('$270^\circ$ cross-section')
# ax.xaxis.set_ticklabels([])
# ax.yaxis.set_ticklabels([])
fig.colorbar(strm.lines, ax=ax.ravel().tolist(), label='$\sqrt{u^2 + w^2}$ [cm/s]')
# axs[2].invert_xaxis()
# axs[3].invert_xaxis()

plt.savefig('streamlines.pgf')

## Velocity Profiles in slice

### Velocity profile of U(z) and W(z) at 90 and 270 degree

In [5]:
fig = plt.figure(constrained_layout=True)
# ax = fig.subplot_mosaic([[0, 1],[2, 3], ['legend', 'legend']], per_subplot_kw={(0, 1): {"suptitle": "log"}})
ax = fig.subplot_mosaic([[0, 1],[2, 3], ['legend', 'legend']], height_ratios = [3.,3.,1.])
plot_x_min = np.inf * np.ones((2,2))
plot_x_max = -np.inf * np.ones((2,2))
plot_y_min = np.inf * np.ones((2,2))
plot_y_max = -np.inf * np.ones((2,2))
sf = ScalarFormatter()
sf.set_scientific(True)
sf.set_powerlimits((0., 0.))

for row in range(7):
    for col in range(2):
        x = X[row][col]
        dx = (x[:,plane_x[col]].max() - x[:,plane_x[col]].min())/Nx
        dz = (x[:,2].max() - x[:,2].min())/Nz
        Z = np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz/2, Nz)
        u = U[row][col]
        v = V[row][col]
        w = W[row][col]
        s = speed[row][col]
        g = grid[row][col]
        ax[col].plot([u, v][plane_x[col]][:,5], Z, label=str(row))
        ax[2+col].plot(w[:,5], Z)
        d_uv = [u, v][plane_x[col]][:,5]
        d_w = w[:,5]
        d_y = Z
        plot_x_min[col, 0] = min(plot_x_min[col, 0], d_uv.min())
        plot_x_max[col, 0] = max(plot_x_max[col, 0], d_uv.max())
        plot_y_min[col, 0] = min(plot_y_min[col, 0], d_y.min())
        plot_y_max[col, 0] = max(plot_y_max[col, 0], d_y.max())
        plot_x_min[col, 1] = min(plot_x_min[col, 1], d_w.min())
        plot_x_max[col, 1] = max(plot_x_max[col, 1], d_w.max())
        plot_y_min[col, 1] = min(plot_y_min[col, 1], d_y.min())
        plot_y_max[col, 1] = max(plot_y_max[col, 1], d_y.max())
for row in range(2):
    for col in range(2):
        ax[2*col + row].set_xticks(list(np.linspace(plot_x_min[row, col], plot_x_max[row, col], 3)))
        ax[2*col + row].set_yticks(list(np.linspace(0, plot_y_max[row, col], 3)))
        ax[2*col + row].set_xticklabels(np.round(list(100*np.linspace(plot_x_min[row, col], plot_x_max[row, col], 3)), 2*col))
        ax[2*col + row].set_yticklabels(np.round(list(100*np.linspace(0, plot_y_max[row, :].max(), 3)),0))
        ax[2*col + row].set_ylim(0-plot_y_max[row, :].max()*0.05, plot_y_max[row, :].max()*1.05)
        ax[2*col + row].grid(visible=True, axis='both', which='major')
        ax[2*col + row].tick_params(axis='both', which='both', length=2., width=0.2)        
        # ax[2*col + row].xaxis.set_major_formatter(deepcopy(sf))
        
        

# ax[1].sharey(ax[0])
# ax[3].sharey(ax[2])
ax[1].set_yticklabels([])
ax[3].set_yticklabels([])
# ax[1].get_xticklabels().visible=False
# ax[1].tick_params('y', labelbottom=False)
# ax[3].tick_params('y', labelbottom=False)

handles, labels = ax[0].get_legend_handles_labels()
# sort_order = [0,4,1,5,2,6,3]
# handles = list(np.array(handles)[sort_order])
# labels = list(np.array(labels)[sort_order])
ax['legend'].legend(handles, labels, ncol=7, mode="expand")
ax['legend'].get_legend().set_title("level")

ax[0].set_title('$90^\circ$ cross-section')
ax[1].set_title('$270^\circ$ cross-section')
ax[0].set_ylabel('$H\; [cm]$')
ax[2].set_ylabel('$H\; [cm]$')

ax[0].set_xlabel('$u\; [cm/s]$')
ax[1].set_xlabel('$u\; [cm/s]$')
ax[2].set_xlabel('$w\; [cm/s]$')
ax[3].set_xlabel('$w\; [cm/s]$')
ax['legend'].axis('off')
# ax[0].set_title('u at $90^\circ$')
# ax[1].set_title('w at $90^\circ$')
# ax[2].set_title('u at $270^\circ$')
# ax[3].set_title('w at $270^\circ$')

plt.savefig('slices_transversal.pgf')

        

### Velocity profile of U(z) at inner and outer wall at 90 and 270 degree

In [6]:
fig = plt.figure(constrained_layout=True)
ax = fig.subplot_mosaic([[0, 1], [2, 3], ["legend", "legend"]], height_ratios = [3., 3., 1.])
plot_x_min = np.inf * np.ones((2, 2))
plot_x_max = -np.inf * np.ones((2, 2))
plot_y_min = np.inf * np.ones((2, 2))
plot_y_max = -np.inf * np.ones((2, 2))
sf = ScalarFormatter()
sf.set_scientific(True)
sf.set_powerlimits((0.0, 0.0))
map_row_col_to_ax = [[2, 0], [1, 3]]
for row in range(7):
    for col in range(2):
        x = X[row][col]
        dx = (x[:, plane_x[col]].max() - x[:, plane_x[col]].min()) / Nx
        dz = (x[:, 2].max() - x[:, 2].min()) / Nz
        Z = np.linspace(x[:, 2].min() + dz / 2, x[:, 2].max() - dz / 2, Nz)
        u = U[row][col]
        v = V[row][col]
        w = W[row][col]
        s = speed[row][col]
        g = grid[row][col]
        ax[map_row_col_to_ax[col][0]].plot(
            [u, v][plane_x[col]][:, 1], Z, label=str(row)
        )
        ax[map_row_col_to_ax[col][1]].plot(
            [u, v][plane_x[col]][:, -2], Z, label=str(row)
        )
        d_uv = [u, v][plane_x[col]][:, 1]
        d_w = [u, v][plane_x[col]][:, -2]
        d_y = Z
        plot_x_min[col, 0] = min(plot_x_min[col, 0], d_uv.min())
        plot_x_max[col, 0] = max(plot_x_max[col, 0], d_uv.max())
        plot_y_min[col, 0] = min(plot_y_min[col, 0], d_y.min())
        plot_y_max[col, 0] = max(plot_y_max[col, 0], d_y.max())
        plot_x_min[col, 1] = min(plot_x_min[col, 1], d_w.min())
        plot_x_max[col, 1] = max(plot_x_max[col, 1], d_w.max())
        plot_y_min[col, 1] = min(plot_y_min[col, 1], d_y.min())
        plot_y_max[col, 1] = max(plot_y_max[col, 1], d_y.max())
for row in range(2):
    for col in range(2):
        ax[2 * col + row].set_xticks(
            list(np.linspace(plot_x_min[row, col], plot_x_max[row, col], 3))
        )
        ax[2 * col + row].set_yticks(list(np.linspace(0, plot_y_max[row, col], 3)))
        ax[2 * col + row].set_xticklabels(
            np.round(
                list(100 * np.linspace(plot_x_min[row, col], plot_x_max[row, col], 3)),
                2,
            )
        )
        ax[2 * col + row].set_yticklabels(
            np.round(list(100 * np.linspace(0, plot_y_max[row, :].max(), 3)), 1)
        )
        ax[2 * col + row].set_ylim(
            0 - plot_y_max[row, :].max() * 0.05, plot_y_max[row, :].max() * 1.05
        )
        ax[2 * col + row].grid(visible=True, axis="both", which="major")
        ax[2 * col + row].tick_params(axis="both", which="both", length=2.0, width=0.2)
        # ax[2*col + row].xaxis.set_major_formatter(deepcopy(sf))
ax[0].set_xlabel("$u_{inner}\; [cm/s]$")
ax[1].set_xlabel("$u_{inner}\; [cm/s]$")
ax[2].set_xlabel("$u_{outer}\; [cm/s]$")
ax[3].set_xlabel("$u_{outer}\; [cm/s]$")
ax["legend"].axis("off")

ax[1].sharex(ax[3])
ax[0].sharex(ax[2])
ax[1].set_yticklabels([])
ax[3].set_yticklabels([])
# ax[1].get_xticklabels().visible=False
# ax[1].tick_params('y', labelbottom=False)
# ax[3].tick_params('y', labelbottom=False)
# ax[0].ticklabel_format(axis='x', style='sci', useOffset=True, scilimits=[0,0])

handles, labels = ax[0].get_legend_handles_labels()
# sort_order = [0,4,1,5,2,6,3]
# handles = list(np.array(handles)[sort_order])
# labels = list(np.array(labels)[sort_order])
ax["legend"].legend(handles, labels, ncol=7, mode="expand")
ax["legend"].get_legend().set_title("level")

# ax[0].set_title('transversal vel. profile (inner , $90^\circ$)')
# ax[1].set_title('transversal vel. profile (outer, $90^\circ$)')
# ax[2].set_title('transversal vel. profile (inner, $270^\circ$)')
# ax[3].set_title('transversal vel. profile (outer, $270^\circ$)')
ax[0].set_title("$90^\circ$ cross-section")
ax[1].set_title("$270^\circ$ cross-section")
ax[0].set_ylabel("inner wall \n $H\; [cm]$")
ax[2].set_ylabel("outer wall \n $H\; [cm]$")

plt.savefig("slices_transversal_walls.pgf")

## Longitudal velocity profiles

In [7]:
fig = plt.figure(constrained_layout=True)
ax = fig.subplot_mosaic([[0, 1],[2, 3], ['legend', 'legend']], height_ratios = [3., 3., 1.])
plot_x_min = np.inf * np.ones((2,2))
plot_x_max = -np.inf * np.ones((2,2))
plot_y_min = np.inf * np.ones((2,2))
plot_y_max = -np.inf * np.ones((2,2))
# map_row_col_to_ax = [[2,0],[1,3]]
for row in range(7):
    for col in range(2):
        x = X[row][col]
        dz = (x[:,2].max() - x[:,2].min())/Nz
        Z = np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz/2, Nz)
        u = U[row][col]
        v = V[row][col]
        w = W[row][col]
        s = speed[row][col]
        g = grid[row][col]

        # if map_row_col_to_ax[col][0] == 0:
        if col == 0:
            # inner wall 90 deg
            x_ax_0 = v[:,2] * 100.
            y_ax_0 = Z * 100.
            r_ax_0 = 0
            c_ax_0 = 0
            #outer wall 90 deg
            x_ax_1 = v[:,-3] * 100.
            y_ax_1 = Z * 100.
            r_ax_1 = 1
            c_ax_1 = 0
        if col == 1:
            # inner wall 270 deg
            x_ax_0 = v[:,2] * 100.
            y_ax_0 = Z * 100.
            r_ax_0 = 0
            c_ax_0 = 1
            # outer wall 270 deg
            x_ax_1 = v[:,-3] * 100.
            y_ax_1 = Z * 100.
            r_ax_1 = 1
            c_ax_1 = 1
            
            
        # ax[map_row_col_to_ax[col][0]].plot([u, v][1-plane_x[col]][:,2] * 100, Z, label=str(row))
        # ax[map_row_col_to_ax[col][1]].plot([u, v][1-plane_x[col]][:,-3] * 100, Z, label=str(row))
        ax[r_ax_0 * 2 + c_ax_0].plot(x_ax_0, y_ax_0, label=str(row))
        ax[r_ax_1 * 2 + c_ax_1].plot(x_ax_1, y_ax_1, label=str(row))
        d_uv = [u, v][1-plane_x[col]][:,2]
        d_w = [u, v][1-plane_x[col]][:,-3]
        d_y = Z
        plot_x_min[r_ax_0, c_ax_0] = min(plot_x_min[r_ax_0, c_ax_0], x_ax_0.min())
        plot_x_max[r_ax_0, c_ax_0] = max(plot_x_max[r_ax_0, c_ax_0], x_ax_0.max())
        plot_y_min[r_ax_0, c_ax_0] = min(plot_y_min[r_ax_0, c_ax_0], y_ax_0.min())
        plot_y_max[r_ax_0, c_ax_0] = max(plot_y_max[r_ax_0, c_ax_0], y_ax_0.max())
        plot_x_min[r_ax_1, c_ax_1] = min(plot_x_min[r_ax_1, c_ax_1], x_ax_1.min())
        plot_x_max[r_ax_1, c_ax_1] = max(plot_x_max[r_ax_1, c_ax_1], x_ax_1.max())
        plot_y_min[r_ax_1, c_ax_1] = min(plot_y_min[r_ax_1, c_ax_1], y_ax_1.min())
        plot_y_max[r_ax_1, c_ax_1] = max(plot_y_max[r_ax_1, c_ax_1], y_ax_1.max())
for row in range(2):
    for col in range(2):
        ax[2*row + col].set_xticks(list(np.round(np.linspace(np.ceil(plot_x_min[row, col]), np.floor(plot_x_max[row, col]), 3), 1)))
        ax[2*row + col].set_yticks(list(np.linspace(0, plot_y_max[row, col], 3)))
        ax[2*row + col].set_xticklabels(np.round(list(np.round(np.linspace(np.ceil(plot_x_min[row, col]), np.floor(plot_x_max[row, col]), 3), 1)), 2))
        ax[2*row + col].set_yticklabels(np.round(list(np.linspace(0, plot_y_max[row, :].max(), 3)),0))
        # ax[2*row + col].set_ylim(0-plot_y_max[row, :].max()*0.05, plot_y_max[row, :].max()*1.05)
        ax[2*row + col].grid(visible=True, axis='both', which='major')
        # ax[2*col + row].tick_params(axis='both', which='both', length=2., width=0.2)
ax[0].set_xlabel('$v_{inner}\; [cm/s]$')
ax[1].set_xlabel('$v_{inner}\; [cm/s]$')
ax[2].set_xlabel('$v_{outer}\; [cm/s]$')
ax[3].set_xlabel('$v_{outer}\; [cm/s]$')
ax['legend'].axis('off')

# ax[1].sharey(ax[0])
# ax[3].sharey(ax[2])
ax[1].set_yticklabels([])
ax[3].set_yticklabels([])
# ax[1].get_xticklabels().visible=False
# ax[1].tick_params('y', labelbottom=False)
# ax[3].tick_params('y', labelbottom=False)

handles, labels = ax[0].get_legend_handles_labels()
# sort_order = [0,4,1,5,2,6,3]
# handles = list(np.array(handles)[sort_order])
# labels = list(np.array(labels)[sort_order])
ax['legend'].legend(handles, labels, ncol=7, mode="expand")
ax['legend'].get_legend().set_title("level")

# ax[0].set_title('longitudinal vel. profile (inner , $90^\circ$)')
# ax[1].set_title('longitudinal vel. profile (outer, $90^\circ$)')
# ax[2].set_title('longitudinal vel. profile (inner, $270^\circ$)')
# ax[3].set_title('longitudinal vel. profile (outer, $270^\circ$)')
ax[0].set_title('$90^\circ$ cross-section')
ax[1].set_title('$270^\circ$ cross-section')
ax[0].set_ylabel('inner wall \n $H\; [cm]$')
ax[2].set_ylabel('outer wall \n $H\; [cm]$')

plt.savefig('slices_longitudinal.pgf')

        

## Vortivity in plane

In [8]:
# fig, ax = plt.subplots(7,2, layout='constrained', figsize=(12,8))
for row in range(1):
    for col in range(1):
        x = X[row][col]
        u = U[row][col]
        v = V[row][col]
        w = W[row][col]
        s = speed[row][col]
        g = grid[row][col]
        dx = (x[:,plane_x[col]].max() - x[:,plane_x[col]].min())/Nx
        dz = (x[:,2].max() - x[:,2].min())/Nz
        x_2d = x[:,0].reshape((Nx, Nz))
        z_2d = x[:,2].reshape((Nx, Nz))
        u_2d = u.reshape((Nx, Nz))
        w_2d = w.reshape((Nx, Nz))
        dudz = np.zeros_like(u_2d)
        dwdx = np.zeros_like(u_2d)
        for i in range(dudz.shape[1]):
            dwdx[:,i] = np.gradient(w_2d[:,i], x_2d[:,i], edge_order=2)
        for i in range(dudz.shape[0]):
            dudz[i,:] = np.gradient(u_2d[i,:], z_2d[i,:], edge_order=2)
        vorticity = -dwdx + dudz
        # plt.contour(x_2d, z_2d, vorticity)
        plt.pcolormesh(x_2d, z_2d, vorticity)
        # seed_points = np.array([x[:,plane_x[col]].mean() * np.ones(N_streamlines), np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz/2, N_streamlines)])
        # ax[row, col].scatter(seed_points[0,:], seed_points[1,:])
        # ax[row, col].streamplot(g[0],g[1],[u, v][plane_x[col]],w, broken_streamlines=False, start_points=seed_points.T, color=s)
        # ax[row, col].set_ylim(x[:,2].min()+dz/2,x[:,2].max()-dz/2)

/tmp/ipykernel_25807/3569931281.py:24: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  plt.pcolormesh(x_2d, z_2d, vorticity)


## Superelevation

In [9]:
fig = plt.figure(constrained_layout=False)
# ax = fig.subplot_mosaic([[0, 1], ['legend', 'legend']])
ax = fig.subplot_mosaic(
    [[0, 1], ["legend", "legend"]],
    per_subplot_kw={
        0: {"box_aspect": 0.6},
        1: {"box_aspect": 0.6},
    },
    height_ratios=[2, 1],
)
plot_x_min = np.inf * np.ones((2))
plot_x_max = -np.inf * np.ones((2))
plot_y_min = np.inf * np.ones((2))
plot_y_max = -np.inf * np.ones((2))
for row in range(7):
    for col in range(2):
        x = X[row][col]
        # dz = (x[:,2].max() - x[:,2].min())/Nz
        # Z = np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz/2, Nz)
        # u = U[row][col]
        # v = V[row][col]
        # w = W[row][col]
        # s = speed[row][col]
        # g = grid[row][col]
        # ax[col].plot([u, v][1-plane_x[col]][:,2], Z, label=str(row))
        # ax[2+col].plot([u, v][1-plane_x[col]][:,-3], Z, label=str(row))
        d_uv = x[Nz-1::Nz, 0]
        d_w = 100*x[Nz-1::Nz, 2]
        x_plot = np.array((d_uv)*100. + (-1)**col* 366)
        ax[col].plot(x_plot, d_w, label=str(row))
        plot_x_min[col] = min((plot_x_min[col]), x_plot.min())
        plot_x_max[col] = max((plot_x_max[col]), x_plot.max())
        plot_y_min[col] = min(plot_y_min[col], d_w.min())
        plot_y_max[col] = max(plot_y_max[col], d_w.max())
for col in range(2):
    ax[col].set_xticks(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)))
    ax[col].set_yticks(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)))
    ax[col].set_xticklabels(np.round(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)), 1))
    ax[col].set_yticklabels(np.round(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)),1))
    ax[col].grid(visible=True, axis='both', which='major')
ax[0].set_xlabel('$x\; [cm]$ \n outer wall$\leftrightarrow$inner wall')
ax[1].set_xlabel('$x\; [cm]$ \n inner wall$\leftrightarrow$outer wall')
ax['legend'].axis('off')

# ax[1].sharey(ax[0])
# ax[3].sharey(ax[2])
# ax[1].set_yticklabels([])

handles, labels = ax[0].get_legend_handles_labels()
ax['legend'].legend(handles, labels, ncol=7, mode="expand",loc='lower left')
ax['legend'].get_legend().set_title("level")

ax[0].set_title('$90^\circ$ cross-section')
ax[1].set_title('$270^\circ$ cross-section')
ax[0].set_ylabel('$H\; [cm]$')


fig.subplots_adjust(bottom=0.3)
plt.savefig('slices_height.pgf')

        

## Create mesh as svg

In [10]:
import meshio 

In [11]:
filename = '../output_cluster/F/streamlines/lvl0/out.0.vtk'
mesh = meshio.read(filename)
# mesh.points = mesh.points[:,:2]
mesh.write('mesh.vtk')

In [12]:
# reader = pv.get_reader('mesh.vtk')
# mesh = reader.read()
# mesh.plot()

In [13]:
pl = pv.Plotter()
_ = pl.add_mesh(mesh, smooth_shading=True, show_edges=True)
# _ = pl.add_background_image(examples.mapfile)
pl.enable_parallel_projection()
pl.show()

Widget(value="<iframe src='http://localhost:42369/index.html?ui=P_0x7fd632e75d60_0&reconnect=auto' style='widt…

In [14]:
pl.save_graphic("mesh.svg")  

# Comparison with Stefflers data

## Height profile

In [15]:
def rescale_data(x, x_scale_data,x_scale_goal):
    x = np.array(x)
    x_out = np.zeros_like(x)
    for i, xp in enumerate(x): 
        x_star = (xp-x_scale_data[0])/(x_scale_data[1]-x_scale_data[0])
        x_out[i] = x_scale_goal[0] + (x_scale_goal[1]-x_scale_goal[0]) * x_star
    return x_out

In [16]:
import notebooks.data_steffler as data

In [17]:
fig = plt.figure(constrained_layout=False)
# ax = fig.subplot_mosaic([[0, 1], ['legend', 'legend']])
ax = fig.subplot_mosaic(
    [[0, 1], ["legend", "legend"]],
    per_subplot_kw={
        0: {"box_aspect": 0.6},
        1: {"box_aspect": 0.6},
    },
    height_ratios=[2, 1],
)
plot_x_min = np.inf * np.ones((2))
plot_x_max = -np.inf * np.ones((2))
plot_y_min = np.inf * np.ones((2))
plot_y_max = -np.inf * np.ones((2))
for row in [0, 1, 6]:
    for col in range(2):
        x = X[row][col]
        d_uv = x[Nz-1::Nz, 0]
        if d_uv[0] > 0:
            d_uv = (d_uv-3.66)*100
        else:
            d_uv = (d_uv+3.66)*100
        d_w = 100*x[Nz-1::Nz, 2]
        ax[col].plot(d_uv, d_w, label='level {}'.format(str(row)))
        plot_x_min[col] = min(plot_x_min[col], d_uv.min())
        plot_x_max[col] = max(plot_x_max[col], d_uv.max())
        plot_y_min[col] = min(plot_y_min[col], d_w.min())
        plot_y_max[col] = max(plot_y_max[col], d_w.max())

data_x, data_y = data.h90_exp()
data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
data_y = np.array(data_y) * 100.
ax[0].plot(data_x, data_y, 'x', label='experiment')
# data_x, data_y = data.h90_VAM()
# data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
# data_y = np.array(data_y) * 100.
# ax[0].plot(data_x, data_y, label='VAM')

plot_x_min[0] = min(plot_x_min[0], np.array(data_x).min())
plot_x_max[0] = max(plot_x_max[0], np.array(data_x).max())
plot_y_min[0] = min(plot_y_min[0], np.array(data_y).min())
plot_y_max[0] = max(plot_y_max[0], np.array(data_y).max())

data_x, data_y = data.h270_exp()
data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
data_y = np.flip(np.array(data_y)) * 100.
ax[1].plot(data_x, data_y, 'x', label='experiment')
# data_x, data_y = data.h270_VAM()
# data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
# data_y = np.flip(np.array(data_y)) * 100.
# ax[1].plot(data_x, data_y, label='VAM')

plot_x_min[1] = min(plot_x_min[1], np.array(data_x).min())
plot_x_max[1] = max(plot_x_max[1], np.array(data_x).max())
plot_y_min[1] = min(plot_y_min[1], np.array(data_y).min())
plot_y_max[1] = max(plot_y_max[1], np.array(data_y).max())

for col in range(2):
    ax[col].set_xticks(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)))
    ax[col].set_yticks(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)))
    ax[col].set_xticklabels(np.round(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)), 1))
    ax[col].set_yticklabels(np.round(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)),1))
    ax[col].grid(visible=True, axis='both', which='major')
ax[0].set_xlabel('$x\; [cm]$ \n outer wall$\leftrightarrow$inner wall')
ax[1].set_xlabel('$x\; [cm]$ \n inner wall$\leftrightarrow$outer wall')
ax['legend'].axis('off')

handles, labels = ax[0].get_legend_handles_labels()

# labels[1] = 'level 6'
ax['legend'].legend(handles, labels, ncol=7, mode="expand", loc='lower left')
# ax['legend'].get_legend().set_title("level")

ax[0].set_title('$90^\circ$ cross-section')
ax[1].set_title('$270^\circ$ cross-section')
ax[0].set_ylabel('$H\; [cm]$')

fig.subplots_adjust(bottom=0.4)
plt.savefig('comp_height.pgf')


## Longitudinal and transversal velocity profile

In [18]:
fig = plt.figure(constrained_layout=False)
ax = fig.subplot_mosaic(
    [[0, 1], ["legend", "legend"]],
    per_subplot_kw={
        0: {"box_aspect": 0.6},
        1: {"box_aspect": 0.6},
    },
    height_ratios=[2, 1],
)
# ax = fig.subplot_mosaic(
#     [[0, 1], ["legend", "legend"]],
# )
plot_x_min = [np.inf, np.inf]
plot_x_max = [-np.inf, -np.inf]
plot_y_min = [np.inf, np.inf]
plot_y_max = [-np.inf, -np.inf]
for col in range(2):
    if col == 1:
        plt.gca().set_prop_cycle(None)
    for row in [0, 1, 6]:
        x = X[row][col]
        dz = (x[:, 2].max() - x[:, 2].min()) / Nz
        # Z = np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz/2, Nz)
        Z = np.linspace(0.0, 1.0, Nz)
        u = U[row][1]
        v = V[row][1]
        w = W[row][1]
        # u_plot = rescale_data(v[:,5], [v[:,5].min(), v[:,5].max()], [0., 1.])
        if col == 0:
            u_plot = np.abs(v[:, -4])
        if col == 1:
            u_plot = np.array(u[:, -2])
        ax[col].plot(u_plot * 100.0, Z, label="level {}".format(str(row)))
        plot_x_min[col] = min(plot_x_min[col], u_plot.min() * 100.0)
        plot_x_max[col] = max(plot_x_max[col], u_plot.max() * 100.0)
        plot_y_min[col] = min(plot_y_min[col], Z.min())
        plot_y_max[col] = max(plot_y_max[col], Z.max())

col = 1
data_x, data_y = data.v_270_m08_exp()
x_plot_exp = np.flip(data_x) * 100.0
y_plot_exp = np.array(data_y)
ax[col].plot(x_plot_exp, y_plot_exp, "x", label="experiment")
data_x, data_y = data.v_270_m08_VAM()
x_plot_VAM = np.flip(data_x) * 100.0
y_plot_VAM = np.array(data_y)
ax[col].plot(x_plot_VAM, y_plot_VAM, label="VAM")
plot_x_min[col] = min(plot_x_min[col], x_plot_exp.min())
plot_x_max[col] = max(plot_x_max[col], x_plot_exp.max())
plot_y_min[col] = min(plot_y_min[col], y_plot_exp.min())
plot_y_max[col] = max(plot_y_max[col], y_plot_exp.max())
plot_x_min[col] = min(plot_x_min[col], x_plot_VAM.min())
plot_x_max[col] = max(plot_x_max[col], x_plot_VAM.max())
plot_y_min[col] = min(plot_y_min[col], y_plot_VAM.min())
plot_y_max[col] = max(plot_y_max[col], y_plot_VAM.max())

col = 0
data_x, data_y = data.u_270_m08_exp()
x_plot_exp = np.array(data_x) * 100.0
y_plot_exp = np.array(data_y)
ax[col].plot(x_plot_exp, y_plot_exp, "x", label="experiment")
data_x, data_y = data.u_270_m08_VAM()
x_plot_VAM = np.array(data_x) * 100.0
y_plot_VAM = np.array(data_y)
ax[col].plot(x_plot_VAM, y_plot_VAM, label="VAM")
plot_x_min[col] = min(plot_x_min[col], x_plot_exp.min())
plot_x_max[col] = max(plot_x_max[col], x_plot_exp.max())
plot_y_min[col] = min(plot_y_min[col], y_plot_exp.min())
plot_y_max[col] = max(plot_y_max[col], y_plot_exp.max())
plot_x_min[col] = min(plot_x_min[col], x_plot_VAM.min())
plot_x_max[col] = max(plot_x_max[col], x_plot_VAM.max())
plot_y_min[col] = min(plot_y_min[col], y_plot_VAM.min())
plot_y_max[col] = max(plot_y_max[col], y_plot_VAM.max())

ax[0].set_xlabel("$|u|\; [cm/s]$")
ax[1].set_xlabel("$v\; [cm/s]$")
ax["legend"].axis("off")

handles, labels = ax[0].get_legend_handles_labels()

# labels[1] = 'level 6'
ax["legend"].legend(handles, labels, ncol=5, mode="expand", loc='lower left')

ax[0].set_title("longitudinal vel. profile ($270^\circ$)")
ax[1].set_title("transversal vel. profile ($270^\circ$)")
ax[0].set_ylabel("$z\; [cm/h]$")
# ax[1].set_ylabel('$z\; [cm/h]$')

ax[1].set_yticklabels([])
for col in range(2):
    ax[col].set_xticks(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)))
    ax[col].set_yticks(list(np.linspace(np.abs(plot_y_min[col]), plot_y_max[col], 3)))
    ax[col].set_xticklabels(
        np.round(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)), 1)
    )
    ax[col].set_yticklabels(
        np.round(list(np.linspace(np.abs(plot_y_min[col]), plot_y_max[col], 3)), 1)
    )
    ax[col].grid(visible=True, axis="both", which="major")
ax[1].set_xticks([plot_x_min[1], 0.0, plot_x_max[1]])
ax[1].set_xticklabels(np.round([plot_x_min[1], 0.0, plot_x_max[1]]))
ax[1].set_yticklabels([])

fig.subplots_adjust(bottom=0.4)
plt.savefig("comp_long_trans.pgf")
plt.savefig("comp_long_trans.png", bbox_inches="tight")

## Longitudinal vel. profile

In [ ]:

fig = plt.figure(constrained_layout=False)
# ax = fig.subplot_mosaic([[0, 1],['legend', 'legend']])
ax = fig.subplot_mosaic(
    [[0, 1], ["legend", "legend"]],
    per_subplot_kw={
        0: {"box_aspect": 0.6},
        1: {"box_aspect": 0.6},
    },
    height_ratios=[2, 1],
)
plot_x_min = [np.inf, np.inf]
plot_x_max = [-np.inf, -np.inf]
plot_y_min = [np.inf, np.inf]
plot_y_max = [-np.inf, -np.inf]
col = 1
for row in [0, 1, 6]:
    x = X[row][col]
    d_uv = x[Nz-1::Nz, 0]
    d_uv = (d_uv-3.66)*100
    # d_w = 100*x[Nz-1::Nz, 2]
    u = U[row][col]
    v = V[row][col]
    w = W[row][col]
    u_cross = np.abs(v[-5,:])*100
    ax[0].plot(d_uv, u_cross, label=str(row))
    dz = (x[:,2].max() - x[:,2].min())/Nz
    Z = np.linspace(0., 1., Nz)
    u_plot = np.abs(v[:,-2] )
    ax[1].plot(u_plot, Z, label='level {}'.format(str(row)))
    plot_x_min[0] = min(plot_x_min[0], d_uv.min())
    plot_x_max[0] = max(plot_x_max[0], d_uv.max())
    plot_y_min[0] = min(plot_y_min[0], u_cross.min())
    plot_y_max[0] = max(plot_y_max[0], u_cross.max())
    plot_x_min[1] = min(plot_x_min[1], u_plot.min())
    plot_x_max[1] = max(plot_x_max[1], u_plot.max())
    plot_y_min[1] = min(plot_y_min[1], Z.min())
    plot_y_max[1] = max(plot_y_max[1], Z.max())

data_x, data_y = data.u_270_crossection_VAM()
data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
ax[0].plot(data_x, np.flip(data_y)*100., label='VAM')
data_x, data_y = data.u_270_crosssection_exp()
data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
ax[0].plot(data_x, np.flip(data_y)*100., 'x', label='experiment')

data_x, data_y = data.u_270_m08_VAM()
ax[1].plot(data_x, np.array(data_y), label='VAM')
data_x, data_y = data.u_270_m08_exp()
ax[1].plot(data_x, np.array(data_y), 'x', label='experiment')


ax[0].set_xlabel('$x\; [cm]$ \n outer wall$\leftrightarrow$inner wall')
ax[1].set_xlabel('$|v| \; [cm/s]$')
ax['legend'].axis('off')

handles, labels = ax[0].get_legend_handles_labels()

# labels[1] = 'level 6'
ax['legend'].legend(handles, labels, ncol=5, mode="expand", loc='lower left')

ax[0].set_title('$270^\circ$ cross-section')
ax[1].set_title('$270^\circ$ longitudinal vel. profile')
ax[0].set_ylabel('$|v|\; [cm/s]$')
ax[1].set_ylabel('$z\; [cm / h]$')

for col in range(2):
    ax[col].set_xticks(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)))
    ax[col].set_yticks(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)))
    ax[col].set_xticklabels(np.round(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)), 1))
    ax[col].set_yticklabels(np.round(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)),1))
    ax[col].grid(visible=True, axis='both', which='major')

plt.savefig('comp_longitudinal.pgf')

## U cross section / longitudinal velocity profile at 270

In [ ]:

fig = plt.figure(constrained_layout=False)
# ax = fig.subplot_mosaic([[0, 1],['legend', 'legend']]:)
ax = fig.subplot_mosaic(
    [[0, 1], ["legend", "legend"]],
    per_subplot_kw={
        0: {"box_aspect": 0.6},
        1: {"box_aspect": 0.6},
    },
    height_ratios=[2, 1],
)
plot_x_min = [np.inf, np.inf]
plot_x_max = [-np.inf, -np.inf]
plot_y_min = [np.inf, np.inf]
plot_y_max = [-np.inf, -np.inf]
col = 1
for row in [0, 1, 6]:
    x = X[row][col]
    d_uv = x[Nz-1::Nz, 0]
    d_uv = (d_uv-3.66)*100
    # d_w = 100*x[Nz-1::Nz, 2]
    u = U[row][col]
    v = V[row][col]
    w = W[row][col]
    u_cross = np.abs(v[-5,:])*100
    ax[0].plot(d_uv, u_cross, label=str(row))
    dz = (x[:,2].max() - x[:,2].min())/Nz
    Z = np.linspace(0., 1., Nz)
    u_plot = np.abs(v[:,-2] )
    ax[1].plot(u_plot, Z, label='level {}'.format(str(row)))
    plot_x_min[0] = min(plot_x_min[0], d_uv.min())
    plot_x_max[0] = max(plot_x_max[0], d_uv.max())
    plot_y_min[0] = min(plot_y_min[0], u_cross.min())
    plot_y_max[0] = max(plot_y_max[0], u_cross.max())
    plot_x_min[1] = min(plot_x_min[1], u_plot.min())
    plot_x_max[1] = max(plot_x_max[1], u_plot.max())
    plot_y_min[1] = min(plot_y_min[1], Z.min())
    plot_y_max[1] = max(plot_y_max[1], Z.max())

data_x, data_y = data.u_270_crossection_VAM()
data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
ax[0].plot(data_x, np.flip(data_y)*100., label='VAM')
data_x, data_y = data.u_270_crosssection_exp()
data_x = rescale_data(data_x, [-1., 1.], [-53.5, 53.5])
ax[0].plot(data_x, np.flip(data_y)*100., 'x', label='experiment')

data_x, data_y = data.u_270_m08_VAM()
ax[1].plot(data_x, np.array(data_y), label='VAM')
data_x, data_y = data.u_270_m08_exp()
ax[1].plot(data_x, np.array(data_y), 'x', label='experiment')


ax[0].set_xlabel('$x\; [cm]$ \n outer wall$\leftrightarrow$inner wall')
ax[1].set_xlabel('$|v| \; [cm/s]$')
ax['legend'].axis('off')

handles, labels = ax[0].get_legend_handles_labels()

# labels[1] = 'level 6'
ax['legend'].legend(handles, labels, ncol=7, mode="expand", loc='lower left')

ax[0].set_title('$270^\circ$ cross-section')
ax[1].set_title('$270^\circ$ longitudinal vel. profile')
ax[0].set_ylabel('$|v|\; [cm/s]$')
ax[1].set_ylabel('$z\; [cm / h]$')

for col in range(2):
    ax[col].set_xticks(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)))
    ax[col].set_yticks(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)))
    ax[col].set_xticklabels(np.round(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)), 1))
    ax[col].set_yticklabels(np.round(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)),1))
    ax[col].grid(visible=True, axis='both', which='major')

fig.subplots_adjust(bottom=0.4)
plt.savefig('comp_longitudinal.pgf')

## Transversal velocity profile

In [ ]:


fig = plt.figure(constrained_layout=True)
ax = fig.subplot_mosaic([[0, 1], ['legend', 'legend']])
plot_x_min = [np.inf, np.inf]
plot_x_max = [-np.inf, -np.inf]
plot_y_min = [np.inf, np.inf]
plot_y_max = [-np.inf, -np.inf]
for col in range(2):
    if col == 1:
        plt.gca().set_prop_cycle(None)
    for row in [0, 1, 6]:
        x = X[row][col]
        dz = (x[:,2].max() - x[:,2].min())/Nz
        # Z = np.linspace(x[:,2].min()+dz/2, x[:,2].max()-dz/2, Nz)
        Z = np.linspace(0., 1., Nz)
        u = U[row][col]
        v = V[row][col]
        w = W[row][col]
        # u_plot = rescale_data(v[:,5], [v[:,5].min(), v[:,5].max()], [0., 1.])
        if col == 0:
            u_plot = u[:,-4] 
        if col == 1:
            u_plot = u[:,-2] 
        u_plot *= 100.
        ax[col].plot(u_plot, Z, label='level {}'.format(str(row)))
        plot_x_min[col] = min(plot_x_min[col], u_plot.min())
        plot_x_max[col] = max(plot_x_max[col], u_plot.max())
        plot_y_min[col] = min(plot_y_min[col], Z.min())
        plot_y_max[col] = max(plot_y_max[col], Z.max())

    if col == 0:
        data_x, data_y = data.v_90_m04_exp()
        x_plot_exp = np.array(data_x)*100.
        y_plot_exp = np.array(data_y)
        ax[0].plot(x_plot_exp, y_plot_exp, 'x', label='experiment')
        data_x, data_y = data.v_90_m04_VAM()
        x_plot_VAM = np.array(data_x)*100.
        y_plot_VAM = np.array(data_y)
        ax[0].plot(x_plot_VAM, y_plot_VAM, label='VAM')
        plot_x_min[col] = min(plot_x_min[col], x_plot_exp.min())
        plot_x_max[col] = max(plot_x_max[col], x_plot_exp.max())
        plot_y_min[col] = min(plot_y_min[col], y_plot_exp.min())
        plot_y_max[col] = max(plot_y_max[col], y_plot_exp.max())
        plot_x_min[col] = min(plot_x_min[col], x_plot_VAM.min())
        plot_x_max[col] = max(plot_x_max[col], x_plot_VAM.max())
        plot_y_min[col] = min(plot_y_min[col], y_plot_VAM.min())
        plot_y_max[col] = max(plot_y_max[col], y_plot_VAM.max())
    if col == 1:
        data_x, data_y = data.v_270_m08_exp()
        x_plot_exp = np.array(data_x)*100.
        y_plot_exp = np.flip(data_y)
        ax[1].plot(x_plot_exp, y_plot_exp, 'x', label='experiment')
        data_x, data_y = data.v_270_m08_VAM()
        x_plot_VAM =np.array(data_x)*100. 
        y_plot_VAM =np.flip(data_y)
        ax[1].plot(x_plot_VAM, y_plot_VAM, label='VAM')
        plot_x_min[col] = min(plot_x_min[col], x_plot_exp.min())
        plot_x_max[col] = max(plot_x_max[col], x_plot_exp.max())
        plot_y_min[col] = min(plot_y_min[col], y_plot_exp.min())
        plot_y_max[col] = max(plot_y_max[col], y_plot_exp.max())
        plot_x_min[col] = min(plot_x_min[col], x_plot_VAM.min())
        plot_x_max[col] = max(plot_x_max[col], x_plot_VAM.max())
        plot_y_min[col] = min(plot_y_min[col], y_plot_VAM.min())
        plot_y_max[col] = max(plot_y_max[col], y_plot_VAM.max())

ax[0].set_xlabel('$u\; [cm/s]$')
ax[1].set_xlabel('$u\; [cm/s]$')
ax['legend'].axis('off')

handles, labels = ax[0].get_legend_handles_labels()

# labels[1] = 'level 6'
ax['legend'].legend(handles, labels, ncol=7, mode="expand")

ax[0].set_title('$90^\circ$ cross-section')
ax[1].set_title('$270^\circ$ cross-section')
ax[0].set_ylabel('$z\; [cm/h]$')
# ax[1].set_ylabel('$z\; [cm/h]$')

ax[1].set_yticklabels([])
for col in range(2):
    ax[col].set_xticks(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)))
    ax[col].set_yticks(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)))
    ax[col].set_xticklabels(np.round(list(np.linspace(plot_x_min[col], plot_x_max[col], 3)), 1))
    ax[col].set_yticklabels(np.round(list(np.linspace(plot_y_min[col], plot_y_max[col], 3)),1))
    ax[col].grid(visible=True, axis='both', which='major')


plt.savefig('comp_transversal.pgf')

## Test plot layout

In [ ]:
# create mosaic figure with custom settings
fig = plt.figure(constrained_layout=False)
ax = fig.subplot_mosaic(
    [[0, 1], ["legend", "legend"]],
    per_subplot_kw={
        # "legend": {"box_aspect": 0.1},
        0: {"box_aspect": 0.4},
        1: {"box_aspect": 0.4},
    },
    height_ratios=[5, 1],
)

# fill dummy data
X = np.linspace(0, 1, 100)
for axis in [0, 1]:
    for line in range(5):
        ax[axis].plot(X, float(line) * X, label="line {}".format(str(line)))


# create legend
handles, labels = ax[0].get_legend_handles_labels()
ax["legend"].legend(handles, labels, ncol=5, mode="expand")
ax["legend"].axis("off")

fig.subplots_adjust(bottom=0.4)
plt.savefig("test.png")